In [1]:
# Add description here
#
# *Note:* You can open this file as a notebook (JupyterLab: right-click on it in the side bar -> Open With -> Notebook)

In [2]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [3]:
# If this task has dependencies, list them them here
# (e.g. upstream = ['some_task']), otherwise leave as None.
upstream = ['s01_get']

# This is a placeholder, leave it as None
product = None
table_name = None

In [4]:
# Parameters
table_name = "house_prices_transformed"
upstream = {"s01_get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/01-extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices_raw.csv", "database": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices.duckdb"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/04-transform.ipynb"}


In [5]:
# your code here...